In [37]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

In [38]:
# Load the dataset
df = pd.read_csv('database.csv', header=0)


In [39]:
X = df[['shap', 'h', 't', 'l', 'fy', 'fc', 'et', 'eb', 'A']]
y = df['n']

In [40]:
# Scale features and target variable to [0, 1]
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
y = scaler.fit_transform(y.values.reshape(-1, 1))

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [42]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer for regression
])


In [43]:
# Compile the model
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=["mae"])

In [44]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_mae', patience=500, restore_best_weights=True)


In [45]:
# Model checkpoint callback
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True)


In [46]:
# TensorBoard callback for profiling
tensorboard = TensorBoard(log_dir="logs/")


In [47]:
# Train the model
model.fit(X_train, y_train, epochs=3000, batch_size=1000, verbose=2, validation_split=0.1, callbacks=[early_stopping, checkpoint, tensorboard])


Epoch 1/3000
3/3 - 1s - loss: 14700.4541 - mae: 0.3890 - val_loss: 111.2011 - val_mae: 0.3403 - 1s/epoch - 352ms/step
Epoch 2/3000
3/3 - 0s - loss: 3165.5447 - mae: 0.3599 - val_loss: 97.1966 - val_mae: 0.3216 - 43ms/epoch - 14ms/step
Epoch 3/3000
3/3 - 0s - loss: 10664.8926 - mae: 0.3453 - val_loss: 91.5070 - val_mae: 0.3187 - 43ms/epoch - 14ms/step
Epoch 4/3000
3/3 - 0s - loss: 7902.6660 - mae: 0.3448 - val_loss: 91.3313 - val_mae: 0.3223 - 41ms/epoch - 14ms/step
Epoch 5/3000
3/3 - 0s - loss: 3488.8809 - mae: 0.3507 - val_loss: 102.8993 - val_mae: 0.3319 - 23ms/epoch - 8ms/step
Epoch 6/3000


/home/max/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3/3 - 0s - loss: 3490.7769 - mae: 0.3600 - val_loss: 110.1705 - val_mae: 0.3362 - 28ms/epoch - 9ms/step
Epoch 7/3000
3/3 - 0s - loss: 5139.1323 - mae: 0.3620 - val_loss: 107.5935 - val_mae: 0.3334 - 25ms/epoch - 8ms/step
Epoch 8/3000
3/3 - 0s - loss: 4009.2358 - mae: 0.3589 - val_loss: 102.5185 - val_mae: 0.3283 - 25ms/epoch - 8ms/step
Epoch 9/3000
3/3 - 0s - loss: 1852.9880 - mae: 0.3524 - val_loss: 93.0286 - val_mae: 0.3193 - 24ms/epoch - 8ms/step
Epoch 10/3000
3/3 - 0s - loss: 3081.2705 - mae: 0.3442 - val_loss: 90.4378 - val_mae: 0.3151 - 42ms/epoch - 14ms/step
Epoch 11/3000
3/3 - 0s - loss: 3260.2551 - mae: 0.3414 - val_loss: 90.4979 - val_mae: 0.3154 - 24ms/epoch - 8ms/step
Epoch 12/3000
3/3 - 0s - loss: 1814.3818 - mae: 0.3433 - val_loss: 97.1352 - val_mae: 0.3197 - 24ms/epoch - 8ms/step
Epoch 13/3000
3/3 - 0s - loss: 2188.3123 - mae: 0.3462 - val_loss: 96.4567 - val_mae: 0.3180 - 25ms/epoch - 8ms/step
Epoch 14/3000
3/3 - 0s - loss: 2689.2146 - mae: 0.3441 - val_loss: 91.3414 - 

In [ ]:
# Evaluate the model
loss, mae = model.evaluate(X_train, y_train)
print("Train Loss:", loss, "Train MAE:", mae)

loss, mae = model.evaluate(X_test, y_test)
print("Test Loss:", loss, "Test MAE:", mae)

In [49]:


# Save the model
model.save('my_model')

# Make predictions
y_pre = model.predict(X_test)
y_pre = [i[0] for i in y_pre]

# Calculate and save errors
error = (y_test - y_pre) / y_test * 100
error = error.sort_values()
error.to_csv('error.csv', index=False)

Training time: 16.18881583213806 seconds
86/86 [==============================] - 0s 921us/step - loss: 1041.0393 - mae: 0.0593
Train Loss: 1041.039306640625 Train MAE: 0.059289343655109406
5/5 [==============================] - 0s 1ms/step - loss: 23.5371 - mae: 0.0691
Test Loss: 23.53708267211914 Test MAE: 0.06914953887462616
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(9, 20), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f5bcc661b40>, 140032780459488), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(9, 20), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f5bcc661b40>, 140032780459488), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(20,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f5c54b36110>, 140035126534560), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(20,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f5c54b36110>, 140035126534560), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(20, 20), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f5c54b6dd80>, 140032782285152), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f5bcc6792a0>, 140035731480192), {}).


INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets


5/5 [==============================] - 0s 860us/step


AttributeError: 'numpy.ndarray' object has no attribute 'sort_values'